<a href="https://colab.research.google.com/github/PaoloMadda01/Projects-with-Google-Colab/blob/main/CODEF3_FC_Ridotti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(GITHUB)

In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg19
from PIL import Image
import torch
import torch.optim as optim
from google.colab import files
from PIL import Image
from torchvision import transforms
import cv2
import numpy as np
from flask import Flask, Response
from IPython.display import display, Image, clear_output
import torchvision.models as models



# Carica il modello VGG-19 pre-addestrato
vgg = models.vgg19(pretrained=True)
vgg = vgg.features.eval().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Usa l'enum dei pesi invece del parametro 'pretrained' deprecato
# vgg = models.vgg19(weights=models.vgg19.VGG19Weights.IMAGENET1K_V1).features


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 112MB/s]


In [ ]:
import torch.nn as nn

class ModifiedVGG19(nn.Module):
    def __init__(self, in_features, reduced_features, num_classes):
        super(ModifiedVGG19, self).__init__()

        # Presumibilmente, la parte convoluzionale del tuo modello rimarrà invariata
        vgg19 = models.vgg19(pretrained=True)
        self.features = vgg19.features

        # Modifica la parte fully connected come descritto nell'articolo
        self.classifier = nn.Sequential(
            nn.Flatten(),  # Strato flat
            nn.Linear(in_features, reduced_features),  # Primo strato FC con parametri ridotti
            nn.ReLU(inplace=True),
            nn.Linear(reduced_features, reduced_features),  # Secondo strato FC con parametri ridotti
            nn.ReLU(inplace=True),
            nn.Linear(reduced_features, num_classes),  # Strato di output senza softmax
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Crea una nuova istanza del modello modificato
model = ModifiedVGG19(in_features=4096, reduced_features=1024, num_classes=1000)
# model = ModifiedVGG19(in_features=256, reduced_features=64, num_classes=2)
vgg = model
model = model.to(device)

In [ ]:
cap = cv2.VideoCapture('video.mp4')


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install cupy-cuda117

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 MB 3.2 MB/s eta 0:00:00


In [ ]:
import cupy as cp
import torch
from torchvision import transforms
from google.colab import files
import imageio
import cv2
from torchvision import transforms
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
from PIL import Image
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F


mse_loss = nn.MSELoss()

# Funzione per caricare l'immagine di stile
def load_image(image_path, transform=None, max_size=None, shape=None):
    image = Image.open(image_path)
    if max_size:
        scale = max_size / max(image.size)
        size = tuple([int(dim * scale) for dim in image.size])
        image = image.resize(size, Image.ANTIALIAS)
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    if transform:
        image = transform(image).unsqueeze(0)
    return image




def gram_matrix(tensor):
    if tensor.dim() == 4:
        b, d, h, w = tensor.size()
    elif tensor.dim() == 3:
        d, h, w = tensor.size()
        b = 1
    else:
        raise TypeError('Tensor ha un numero di dimensioni non supportato.')

    tensor = tensor.view(b * d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram


def style_loss(style, target):
    # Assicurati che il tensore target abbia la stessa dimensione del tensore style
    target_resized = F.interpolate(target, size=style.shape[-2:], mode='bilinear', align_corners=False)
    style_gram = gram_matrix(style)
    target_gram = gram_matrix(target_resized)
    return mse_loss(style_gram, target_gram)

def content_loss(content, target):
    # Assicurati che il tensore target abbia la stessa dimensione del tensore content
    target_resized = F.interpolate(target, size=content.shape[-2:], mode='bilinear', align_corners=False)
    return mse_loss(content, target_resized)




def prune_model(model, pruning_rate):
    # Pruning del modello
    for name, module in model.named_modules():
        # Pruning del 50% delle connessioni in ogni strato convoluzionale
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)

    # Rimozione delle maschere di pruning (opzionale)
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.remove(module, 'weight')



def style_transfer(content_image, style_image, model, iterations, content_weight, style_weight):
    content_image_shape = content_image.shape[-2:]


    style_image_tr = resize_style_image(style_image, content_image_shape)      # Da spostare fuori dal while???

    content_image = content_image.to(device)
    style_image = style_image.to(device)
    model = model.eval()  # Aggiunta della modalità di valutazione

    # Usa gli indici degli strati corretti per VGG-19
    content_layers = ['conv4_2']
    style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']

    content_features = get_features(content_image, model)
    style_features = get_features(style_image_tr, model)

    input_image = content_image.clone().requires_grad_(True)

    optimizer = optim.Adam([input_image.requires_grad_()], lr=0.01)

    scaler = GradScaler()  # Aggiunto per la precisione mista

    for i in range(iterations):
        optimizer.zero_grad()

        with autocast():  # Aggiunto per la precisione mista
            input_features = get_features(input_image, model)
            print(f'input_features[content_layers[0]].shape: {input_features[content_layers[0]].shape}')  # Nuova riga
            print(f'content_features[content_layers[0]].shape: {content_features[content_layers[0]].shape}')  # Nuova riga
            c_loss = content_loss(input_features[content_layers[0]], content_features[content_layers[0]])

            s_loss = 0
            for layer in style_layers:
                print(f'input_features[layer].shape: {input_features[layer].shape}')  # Nuova riga
                print(f'style_features[layer].shape: {style_features[layer].shape}')  # Nuova riga
                s_loss += style_loss(input_features[layer], style_features[layer])

            total_loss = content_weight * c_loss + style_weight * s_loss

        scaler.scale(total_loss).backward(retain_graph=True)  # Aggiunto per la precisione mista
        scaler.step(optimizer)  # Aggiunto per la precisione mista
        scaler.update()  # Aggiunto per la precisione mista

    return input_image




def get_features(image, model, layers=None):
    if layers is None:
        layers = {
            '0': 'conv1_1',
            '5': 'conv2_1',
            '10': 'conv3_1',
            '19': 'conv4_1',
            '21': 'conv4_2',  # questo è il layer di contenuto
            '28': 'conv5_1'
        }

    features = {}
    x = image
    for name, layer in model.features.named_children():  # Modifica qui per iterare attraverso gli strati di model.features
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features


def resize_style_image(st_image, target_shape):
    if isinstance(st_image, torch.Tensor):
        st_image = transforms.ToPILImage()(st_image)
    transform = transforms.Compose([
        transforms.Resize(target_shape, antialias=True),  # specifica antialias=True per evitare l'avviso
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(st_image)


def resize_image(image, size=(224, 224)):
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
    ])
    return transform(image)


def trasform_frame(input_image):
    input_image = input_image.squeeze(0).permute(1, 2, 0).cpu().numpy()                # Un po' lento
    input_image = input_image * 255  # Scala i valori dei pixel al range [0, 255]
    return input_image




writer = imageio.get_writer('styled_video.mp4', fps=60)

frame_count = 0
start_time = time.time()

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

batch_frames = []
batch_size = 4

# Prune il modello prima di entrare nel loop di elaborazione video
pruning_rate = 0.5    # 50% pruning
prune_model(model, pruning_rate)

exit_loop = False

while not exit_loop:  # Modifica la condizione del ciclo while per controllare la variabile flag
    for _ in range(batch_size):
        ret, frame = cap.read()
        if not ret:
            exit_loop = True  # Imposta la variabile flag a True se ret è False
            break  # Esce dal ciclo for

        # Converti il frame in tensor
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)
        content = resize_image(frame_pil).unsqueeze(0).to('cuda')

        batch_frames.append(content)

    if not batch_frames:
        break

    batch_content = torch.cat(batch_frames, dim=0)  # Crea un batch
    batch_frames.clear()  # Svuota la lista per il prossimo batc

    style_image = style_image_pil.transform

    output = style_transfer(batch_content, style_image, model, iterations=20, content_weight=1, style_weight=1e6)


    output_frame_np = trasform_frame(output.detach())  # Nota: ora stiamo chiamando la nuova funzione trasform_frame

    # Normalizza l'immagine nell'intervallo 0-255 e convertila in uint8
    output_frame_np = ((output_frame_np - output_frame_np.min()) / (output_frame_np.max() - output_frame_np.min()) * 255).astype(np.uint8)

    writer.append_data(output_frame_np)  # Scrivi il frame sul disco usando imageio

    frame_count += 1
    print(frame_count)

    # Calcola e stampa i FPS ogni secondo
    elapsed_time = time.time() - start_time
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        print(f'FPS: {fps:.2f}')
        # Resetta il contatore dei frame e il tempo di inizio per il prossimo intervallo
        frame_count = 0
        start_time = time.time()



# Chiudi il writer
writer.close()

# Scarica il video elaborato
files.download('styled_video.mp4')



TypeError: ignored

In [ ]:
from PIL import Image

# Carica l'immagine dal tuo computer
uploaded = files.upload()
style_image_path = list(uploaded.keys())[0]
style_image_pil = Image.open(style_image_path)


Saving URLO.jpeg to URLO.jpeg


In [ ]:
from google.colab import files
import cv2
import numpy as np

# Carica il video
uploaded = files.upload()
video_name = list(uploaded.keys())[0]

# Salva il video in una variabile
video_data = uploaded[video_name]

# Scrivi il video su disco temporaneamente per poterlo utilizzare con cv2
with open("video.mp4", "wb") as f:
  f.write(video_data)

Saving video.mp4 to video.mp4


------

------

In [ ]:
import os

num_cores = os.cpu_count()
print(f"Numero di core della CPU: {num_cores}")


Numero di core della CPU: 2


In [ ]:
print(type(output))
print(output)



# Converti l'immagine PIL in un array NumPy
output_array = np.array(output)

# Converti l'array da RGB a BGR (formato OpenCV)
output_frame = cv2.cvtColor(output_array, cv2.COLOR_RGB2BGR)

# Salva il frame elaborato
out.write(output_frame)

NameError: ignored

In [ ]:
from google.colab import files

# Salva il frame come immagine
cv2.imwrite('output_frame.jpg', output_frame)

# Scarica il file
files.download('output_frame.jpg')


NameError: ignored